In [10]:
import torch

### 1.简单例子

In [11]:
x = torch.arange(4.0)
x,x.dtype

(tensor([0., 1., 2., 3.]), torch.float32)

In [12]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  #默认值是None

In [13]:
print(x.grad )

None


In [14]:
y = 2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [15]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [7]:
x.grad == 4 * x

tensor([True, True, True, True])

In [8]:
y = x.sum()
y.backward()
x.grad

tensor([ 1.,  5.,  9., 13.])

In [16]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

### 2.非标量变量的反向传播

In [45]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

In [46]:
y

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

### 3.分离计算

这里可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息。 换句话说，梯度不会向后流经u到x。 因此，下面的反向传播函数计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数。

In [48]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x
x,y,u,z

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>),
 tensor([0., 1., 4., 9.]),
 tensor([ 0.,  1.,  8., 27.], grad_fn=<MulBackward0>))

In [49]:
z.sum().backward()

In [50]:
x.grad

tensor([0., 1., 4., 9.])

In [52]:
x.grad == u

tensor([True, True, True, True])

---------------

In [53]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### 4. Python控制流的梯度计算

In [65]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [66]:
a = torch.randn(size=(), requires_grad=True)
a

tensor(0.1393, requires_grad=True)

In [67]:
d = f(a)
d

tensor(1141.1554, grad_fn=<MulBackward0>)

In [68]:
d.backward()

In [69]:
a.grad

tensor(8192.)

In [70]:
a.grad == d / a

tensor(True)